In [1]:
from bs4 import BeautifulSoup
# import datetime
import numpy as np
import pandas as pd
# from requests import get
# from time import sleep
from urllib.request import urlopen
# import urllib3
# import json


In [2]:
html = urlopen('https://cleaningtheglass.com/stats/league/summary')
soup = BeautifulSoup(html, 'html.parser')
table = soup.find("table")
df_ctg = pd.read_html(str(table), flavor='lxml')[0]
df_ctg.columns = df_ctg.columns.get_level_values(1)
df_ctg = df_ctg.iloc[: , 1:]
df_ctg_1 = df_ctg['Team']
df_ctg_2 = df_ctg.iloc[: , 2:7]
df_ctg = pd.concat([df_ctg_1, df_ctg_2], axis=1)

/var/folders/xl/0gbs3t9n2m92zzqwh0x7vkx00000gp/T/ipykernel_60703/2333859409.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_ctg = pd.read_html(str(table), flavor='lxml')[0]


In [3]:
team_names = {
    'full_name':[
        'Atlanta',
        'Boston',
        'Brooklyn',
        'Charlotte',
        'Chicago',
        'Cleveland',
        'Dallas',
        'Denver',
        'Detroit',
        'Golden State',
        'Houston',
        'Indiana',
        'LA Clippers',
        'LA Lakers',
        'Memphis',
        'Miami',
        'Milwaukee',
        'Minnesota',
        'New Orleans',
        'New York',
        'Oklahoma City',
        'Orlando',
        'Philadelphia',
        'Phoenix',
        'Portland',
        'Sacramento',
        'San Antonio',
        'Toronto',
        'Utah',
        'Washington'
        ],
    'abbr_name':[
        'ATL',
        'BOS',
        'BRK',
        'CHO',
        'CHI',
        'CLE',
        'DAL',
        'DEN',
        'DET',
        'GSW',
        'HOU',
        'IND',
        'LAC',
        'LAL',
        'MEM',
        'MIA',
        'MIL',
        'MIN',
        'NOP',
        'NYK',
        'OKC',
        'ORL',
        'PHI',
        'PHO',
        'POR',
        'SAC',
        'SAS',
        'TOR',
        'UTA',
        'WAS'
        ]
}

team_name_map = pd.DataFrame(team_names)

df_ctg = pd.merge(df_ctg, team_name_map, how='inner', left_on ='Team', right_on='full_name')

del df_ctg['Team']
del df_ctg['full_name']

In [4]:
df_ctg = df_ctg.rename(columns={
    'abbr_name':'TEAM',
    'Exp W82':'Expected Wins (per Point Diff)',
    'W':'Wins',
    'L':'Losses'
})
df_ctg['Expected Wins (per Win%)'] = round(((df_ctg['Win%'].str.rstrip('%').astype('float') / 100.0) * 82),1)


In [5]:
team_tracker = pd.read_csv('support/23_24_team_tracker_template.csv')

In [6]:
team_tracker_results = pd.merge(team_tracker, df_ctg, how='inner', on='TEAM')

team_tracker_results_cols = [
    'TEAM',
    'Over/Under',
    'Wins',
    'Losses',
    'Win%',
    'Point Diff',
    'Expected Wins (per Win%)',
    'Expected Wins (per Point Diff)',
    'Expected O/U (per Win%)',
    'Expected O/U (per Point Diff)'
]

team_tracker_results['Expected O/U (per Win%)'] = np.where(
    team_tracker_results['Expected Wins (per Win%)'] > team_tracker_results['Over/Under'], 
    'Over',
    np.where(
        team_tracker_results['Expected Wins (per Win%)'] < team_tracker_results['Over/Under'], 'Under','Push'
    )
)

team_tracker_results['Expected O/U (per Point Diff)'] = np.where(
    team_tracker_results['Expected Wins (per Point Diff)'] > team_tracker_results['Over/Under'], 
    'Over',
    np.where(
        team_tracker_results['Expected Wins (per Point Diff)'] < team_tracker_results['Over/Under'], 'Under','Push'
    )
)

team_tracker_results = team_tracker_results[team_tracker_results_cols]
team_tracker_results.to_csv('23_24_team_tracker_results.csv')

In [9]:
### Tab 3 entires
ou_picks = pd.read_csv('support/23_24_ou_picks.csv')
ou_picks_for_tab = ou_picks[ou_picks['Full name'] != 'Over Under Line']

,Full name,Boston Celtics,Milwaukee Bucks,Denver Nuggets,Phoenix Suns,Cleveland Cavaliers,Philadelphia 76ers,GS Warriors,LA Lakers,LA Clippers,...,Houston Rockets,Charlotte Hornets,San Antonio Spurs,Portland Trail Blazers,Detroit Pistons,Washington Wizards,Moneyball: Big (3.5),Moneyball: 1-Small (1.5),Moneyball: 2-Small (1.5),Moneyball: 3-Small (1.5)
1,Blake Fahey,Under,Over,Over,Under,Under,Under,Over,Under,Under,...,Under,Under,Under,Over,Over,Over,GS Warriors,Boston Celtics,Milwaukee Bucks,Denver Nuggets
2,Alanna Hulburd,Over,Over,Over,Over,Under,Under,Over,Under,Over,...,Under,Under,Over,Under,Under,Over,GS Warriors,Boston Celtics,Phoenix Suns,Denver Nuggets
3,Michael Hoff,Over,Under,Over,Over,Over,Over,Under,Over,Under,...,Under,Under,Under,Under,Under,Under,Atlanta Hawks,Memphis Grizzlies,Orlando Magic,Indiana Pacers
4,kyle,Over,Over,Over,Under,Over,Under,Over,Under,Under,...,Under,Over,Under,Under,Under,Over,GS Warriors,LA Lakers,Toronto Raptors,OKC Thunder
5,grass guibert,Over,Under,Over,Under,Under,Over,Over,Under,Under,...,Over,Over,Under,Under,Over,Under,GS Warriors,Boston Celtics,Denver Nuggets,Chicago Bulls
6,Johnny curran,Under,Over,Over,Under,Under,Under,Under,Over,Over,...,Over,Under,Under,Under,Over,Over,LA Clippers,Philadelphia 76ers,Memphis Grizzlies,Minnesota Timberwolves
7,Brandon Greene,Over,Over,Under,Over,Under,Under,Over,Over,Under,...,Under,Under,Over,Under,Under,Over,Denver Nuggets,GS Warriors,Portland Trail Blazers,San Antonio Spurs
8,Michael Marzec,Over,Under,Over,Over,Over,Under,Over,Under,Under,...,Under,Over,Under,Under,Under,Over,Minnesota Timberwolves,Phoenix Suns,Detroit Pistons,Utah Jazz
9,Tucker,Under,Over,Over,Over,Under,Over,Under,Under,Over,...,Over,Under,Over,Over,Under,Under,Minnesota Timberwolves,BKN Nets,NO Pelicans,San Antonio Spurs
10,Danny Swift,Over,Over,Over,Under,Over,Over,Over,Under,Under,...,Under,Under,Under,Under,Over,Over,NY Knicks,Philadelphia 76ers,Miami Heat,Washington Wizards


In [10]:
### Tab 1 Prep

In [11]:
ou_picks

,Full name,Boston Celtics,Milwaukee Bucks,Denver Nuggets,Phoenix Suns,Cleveland Cavaliers,Philadelphia 76ers,GS Warriors,LA Lakers,LA Clippers,...,Houston Rockets,Charlotte Hornets,San Antonio Spurs,Portland Trail Blazers,Detroit Pistons,Washington Wizards,Moneyball: Big (3.5),Moneyball: 1-Small (1.5),Moneyball: 2-Small (1.5),Moneyball: 3-Small (1.5)
0,Over Under Line,55.5,54.5,52.5,51.5,50.5,49.5,47.5,47.5,46.5,...,31.5,31.5,29.5,28.5,27.5,24.5,NaN,NaN,NaN,NaN
1,Blake Fahey,Under,Over,Over,Under,Under,Under,Over,Under,Under,...,Under,Under,Under,Over,Over,Over,GS Warriors,Boston Celtics,Milwaukee Bucks,Denver Nuggets
2,Alanna Hulburd,Over,Over,Over,Over,Under,Under,Over,Under,Over,...,Under,Under,Over,Under,Under,Over,GS Warriors,Boston Celtics,Phoenix Suns,Denver Nuggets
3,Michael Hoff,Over,Under,Over,Over,Over,Over,Under,Over,Under,...,Under,Under,Under,Under,Under,Under,Atlanta Hawks,Memphis Grizzlies,Orlando Magic,Indiana Pacers
4,kyle,Over,Over,Over,Under,Over,Under,Over,Under,Under,...,Under,Over,Under,Under,Under,Over,GS Warriors,LA Lakers,Toronto Raptors,OKC Thunder
5,grass guibert,Over,Under,Over,Under,Under,Over,Over,Under,Under,...,Over,Over,Under,Under,Over,Under,GS Warriors,Boston Celtics,Denver Nuggets,Chicago Bulls
6,Johnny curran,Under,Over,Over,Under,Under,Under,Under,Over,Over,...,Over,Under,Under,Under,Over,Over,LA Clippers,Philadelphia 76ers,Memphis Grizzlies,Minnesota Timberwolves
7,Brandon Greene,Over,Over,Under,Over,Under,Under,Over,Over,Under,...,Under,Under,Over,Under,Under,Over,Denver Nuggets,GS Warriors,Portland Trail Blazers,San Antonio Spurs
8,Michael Marzec,Over,Under,Over,Over,Over,Under,Over,Under,Under,...,Under,Over,Under,Under,Under,Over,Minnesota Timberwolves,Phoenix Suns,Detroit Pistons,Utah Jazz
9,Tucker,Under,Over,Over,Over,Under,Over,Under,Under,Over,...,Over,Under,Over,Over,Under,Under,Minnesota Timberwolves,BKN Nets,NO Pelicans,San Antonio Spurs


In [12]:
ou_picks_by_team = ou_picks.copy()
del ou_picks_by_team['Moneyball: Big (3.5)']
del ou_picks_by_team['Moneyball: 1-Small (1.5)']
del ou_picks_by_team['Moneyball: 2-Small (1.5)']
del ou_picks_by_team['Moneyball: 3-Small (1.5)']

ou_picks_moneyball = ou_picks.copy()
ou_picks_moneyball = ou_picks_moneyball[['Full name','Moneyball: Big (3.5)','Moneyball: 1-Small (1.5)','Moneyball: 2-Small (1.5)','Moneyball: 3-Small (1.5)']]

In [13]:
predictions = team_tracker_results[['TEAM','Expected O/U (per Win%)','Expected O/U (per Point Diff)']]

In [14]:
team_names = {
    'Long Name':[
        'Atlanta Hawks',
        'BKN Nets',
        'Boston Celtics',
        'Charlotte Hornets',
        'Chicago Bulls',
        'Cleveland Cavaliers',
        'Dallas Mavericks',
        'Denver Nuggets',
        'Detroit Pistons',
        'GS Warriors',
        'Houston Rockets',
        'Indiana Pacers',
        'LA Clippers',
        'LA Lakers',
        'Memphis Grizzlies',
        'Miami Heat',
        'Milwaukee Bucks',
        'Minnesota Timberwolves',
        'NO Pelicans',
        'NY Knicks',
        'OKC Thunder',
        'Orlando Magic',
        'Philadelphia 76ers',
        'Phoenix Suns',
        'Portland Trail Blazers',
        'Sacramento Kings',
        'San Antonio Spurs',
        'Toronto Raptors',
        'Utah Jazz',
        'Washington Wizards'
        ],
    'TEAM':[
        'ATL',
        'BOS',
        'BRK',
        'CHO',
        'CHI',
        'CLE',
        'DAL',
        'DEN',
        'DET',
        'GSW',
        'HOU',
        'IND',
        'LAC',
        'LAL',
        'MEM',
        'MIA',
        'MIL',
        'MIN',
        'NOP',
        'NYK',
        'OKC',
        'ORL',
        'PHI',
        'PHO',
        'POR',
        'SAC',
        'SAS',
        'TOR',
        'UTA',
        'WAS'
        ]
}
team_name_map = pd.DataFrame(team_names)

In [15]:
predictions_rename = pd.merge(predictions,team_name_map,on='TEAM',how='inner')
del predictions_rename['TEAM']
predictions_rename = predictions_rename.rename(columns={'Long Name':'TEAM'})

In [16]:
ou_picks_by_team_t = ou_picks_by_team.T
ou_picks_by_team_t.columns = ou_picks_by_team_t.iloc[0]
ou_picks_by_team_t = ou_picks_by_team_t[ou_picks_by_team_t['Over Under Line'] != 'Over Under Line'].reset_index()
ou_picks_by_team_t = ou_picks_by_team_t.rename(columns={'index':'TEAM'})

In [18]:
prep_results_win_percent = pd.merge(ou_picks_by_team_t, predictions_rename[['TEAM','Expected O/U (per Win%)']], on='TEAM',how='inner')

In [19]:
entrants_col = list(prep_results_win_percent.columns)
remove_fields = ['TEAM','Over Under Line','Expected O/U (per Win%)','Expected O/U (per Point Diff)']
final_entrants = [x for x in entrants_col if x not in remove_fields]

In [20]:
for person in final_entrants:
    prep_results_win_percent[person] = np.where(prep_results_win_percent[person] == prep_results_win_percent['Expected O/U (per Win%)'],1,0)
prep_results_win_percent_t = prep_results_win_percent.T
prep_results_win_percent_t['Expected Correct (per Win%)'] = prep_results_win_percent_t.sum(axis=1)
prep_results_win_percent_t = prep_results_win_percent_t.reset_index()
prep_results_win_percent_t = prep_results_win_percent_t.rename(columns={'index':'Name'})
prep_results_win_percent_t = prep_results_win_percent_t[~prep_results_win_percent_t['Name'].isin(['TEAM','Over Under Line','Expected O/U (per Win%)'])]
prep_results_win_percent_t = prep_results_win_percent_t[['Name','Expected Correct (per Win%)']].reset_index(drop=True)

prep_results_point_diff = pd.merge(ou_picks_by_team_t, predictions_rename[['TEAM','Expected O/U (per Point Diff)']], on='TEAM',how='inner')
for person in final_entrants:
    prep_results_point_diff[person] = np.where(prep_results_point_diff[person] == prep_results_point_diff['Expected O/U (per Point Diff)'],1,0)
prep_results_point_diff_t = prep_results_point_diff.T
prep_results_point_diff_t['Expected Correct (per Point Diff)'] = prep_results_point_diff_t.sum(axis=1)
prep_results_point_diff_t = prep_results_point_diff_t.reset_index()
prep_results_point_diff_t = prep_results_point_diff_t.rename(columns={'index':'Name'})
prep_results_point_diff_t = prep_results_point_diff_t[~prep_results_point_diff_t['Name'].isin(['TEAM','Over Under Line','Expected O/U (per Point Diff)'])]
prep_results_point_diff_t = prep_results_point_diff_t[['Name','Expected Correct (per Point Diff)']].reset_index(drop=True)      

prep_results_winner = pd.merge(prep_results_win_percent_t, prep_results_point_diff_t, on='Name',how='inner')

In [22]:
ou_picks_moneyball_prep = ou_picks_moneyball[ou_picks_moneyball['Full name'] != 'Over Under Line']

In [42]:
def moneyball_predictions(moneyball_column, winp_or_pointdiff, output_column, big_or_small):
    ou_picks_moneyball_prep_df = ou_picks_moneyball_prep[['Full name',moneyball_column]]

    # ugly iteration but it will work for now
    moneyball_df = pd.DataFrame()
    for name in ou_picks_moneyball_prep_df['Full name']:
        selected_team = ou_picks_moneyball_prep_df[ou_picks_moneyball_prep_df['Full name']==name][moneyball_column].reset_index(drop=True).iloc[0]
        ou_pick = ou_picks_by_team[ou_picks_by_team['Full name'] == name][selected_team].reset_index(drop=True).iloc[0]
        df_row = pd.DataFrame({'Full name':name,'TEAM':selected_team,'Pick':ou_pick},index=[0])
        moneyball_df = pd.concat([moneyball_df,df_row])

    moneyball_df = pd.merge(moneyball_df,predictions_rename[['TEAM',winp_or_pointdiff]],on='TEAM',how='inner')
    
    if big_or_small == 'Big':
        moneyball_df[output_column] = np.where(moneyball_df['Pick'] == moneyball_df[winp_or_pointdiff],3.5,0)
    elif big_or_small == 'Small':
        moneyball_df[output_column] = np.where(moneyball_df['Pick'] == moneyball_df[winp_or_pointdiff],1.5,0)
    else:
        print('big v small error')

    moneyball_df = moneyball_df[['Full name',output_column]]
    
    return moneyball_df



In [56]:
# Moneyball: Win %
moneyball_big_df_winp = moneyball_predictions('Moneyball: Big (3.5)', 'Expected O/U (per Point Diff)','Moneyball - Big: Point Diff','Big')
moneyball_small1_df_winp = moneyball_predictions('Moneyball: 1-Small (1.5)', 'Expected O/U (per Point Diff)','Moneyball - Small1: Point Diff','Small')
moneyball_small2_df_winp = moneyball_predictions('Moneyball: 2-Small (1.5)', 'Expected O/U (per Point Diff)','Moneyball - Small2: Point Diff','Small')
moneyball_small3_df_winp = moneyball_predictions('Moneyball: 3-Small (1.5)', 'Expected O/U (per Point Diff)','Moneyball - Small3: Point Diff','Small')

# Moneyball: Point Diff
moneyball_big_df_point_diff = moneyball_predictions('Moneyball: Big (3.5)', 'Expected O/U (per Point Diff)','Moneyball - Big: Point Diff','Big')
moneyball_small1_df_point_diff = moneyball_predictions('Moneyball: 1-Small (1.5)', 'Expected O/U (per Point Diff)','Moneyball - Small1: Point Diff','Small')
moneyball_small2_df_point_diff = moneyball_predictions('Moneyball: 2-Small (1.5)', 'Expected O/U (per Point Diff)','Moneyball - Small2: Point Diff','Small')
moneyball_small3_df_point_diff = moneyball_predictions('Moneyball: 3-Small (1.5)', 'Expected O/U (per Point Diff)','Moneyball - Small3: Point Diff','Small')

In [81]:
mb_winp_final = pd.merge(moneyball_big_df_winp, moneyball_small1_df_winp, on='Full name',how='inner')
mb_winp_final = pd.merge(mb_winp_final, moneyball_small2_df_winp, on='Full name',how='inner')
mb_winp_final = pd.merge(mb_winp_final, moneyball_small3_df_winp, on='Full name',how='inner')
# mb_winp_final['Expected Bonus Points (per Win%)'] = point_diff_mb_final.sum(axis=1)
mb_winp_final.index = mb_winp_final['Full name']
del mb_winp_final['Full name']
mb_winp_final['Expected Bonus Points (per Win%)'] = mb_winp_final.sum(axis=1)
mb_winp_final = mb_winp_final.reset_index()
mb_winp_final = mb_winp_final[['Full name','Expected Bonus Points (per Win%)']]
mb_winp_final = mb_winp_final.rename(columns={'Full name':'Name'})

In [80]:
mb_pointdiff_final = pd.merge(moneyball_big_df_point_diff, moneyball_small1_df_point_diff, on='Full name',how='inner')
mb_pointdiff_final = pd.merge(mb_pointdiff_final, moneyball_small2_df_point_diff, on='Full name',how='inner')
mb_pointdiff_final = pd.merge(mb_pointdiff_final, moneyball_small3_df_point_diff, on='Full name',how='inner')
mb_pointdiff_final.index = mb_pointdiff_final['Full name']
del mb_pointdiff_final['Full name']
mb_pointdiff_final['Expected Bonus Points (per Point Diff)'] = mb_pointdiff_final.sum(axis=1)
mb_pointdiff_final = mb_pointdiff_final.reset_index()
mb_pointdiff_final = mb_pointdiff_final[['Full name','Expected Bonus Points (per Point Diff)']]
mb_pointdiff_final = mb_pointdiff_final.rename(columns={'Full name':'Name'})

In [98]:
## Expected Correct
prep_results_winner_final = pd.merge(prep_results_winner,mb_winp_final,on='Name',how='inner')
prep_results_winner_final = pd.merge(prep_results_winner_final,mb_pointdiff_final,on='Name',how='inner')
prep_results_winner_final['Expected Total Score (Per Win%)'] = prep_results_winner_final['Expected Correct (per Win%)'] + prep_results_winner_final['Expected Bonus Points (per Win%)']
prep_results_winner_final['Expected Total Score (Per Point Diff)'] = prep_results_winner_final['Expected Correct (per Point Diff)'] + prep_results_winner_final['Expected Bonus Points (per Point Diff)']

prep_results_winner_final = prep_results_winner_final.sort_values('Expected Total Score (Per Point Diff)',ascending=False).reset_index(drop=True)
prep_results_winner_final = prep_results_winner_final.reset_index().rename(columns={'index':'Expected Result (per Point Diff)'})
prep_results_winner_final['Expected Result (per Point Diff)'] = prep_results_winner_final['Expected Result (per Point Diff)'] + 1

prep_results_winner_final = prep_results_winner_final.sort_values('Expected Total Score (Per Win%)',ascending=False).reset_index(drop=True)
prep_results_winner_final = prep_results_winner_final.reset_index().rename(columns={'index':'Expected Result (per Win%)'})
prep_results_winner_final['Expected Result (per Win%)'] = prep_results_winner_final['Expected Result (per Win%)'] + 1

result_col_order = [
    'Name',
    'Expected Correct (per Win%)',
    'Expected Correct (per Point Diff)',
    'Expected Bonus Points (per Win%)',
    'Expected Bonus Points (per Point Diff)',
    'Expected Total Score (Per Win%)',
    'Expected Total Score (Per Point Diff)',
    'Expected Result (per Win%)',
    'Expected Result (per Point Diff)',
]

prep_results_winner_final = prep_results_winner_final[result_col_order]
prep_results_winner_final


,Name,Expected Correct (per Win%),Expected Correct (per Point Diff),Expected Bonus Points (per Win%),Expected Bonus Points (per Point Diff),Expected Total Score (Per Win%),Expected Total Score (Per Point Diff),Expected Result (per Win%),Expected Result (per Point Diff)
0,Johnny curran,17,17,6.5,6.5,23.5,23.5,1,2
1,Ike Chukwu,14,19,8.0,8.0,22.0,27.0,2,1
2,Danny Swift,17,12,5.0,5.0,22.0,17.0,3,10
3,Michael Hoff,18,18,3.5,3.5,21.5,21.5,4,4
4,Mike Halloran,18,18,3.0,3.0,21.0,21.0,5,6
5,Luke Serra,16,18,5.0,5.0,21.0,23.0,6,3
6,Michael Andrew Ricciardulli,19,13,1.5,1.5,20.5,14.5,7,13
7,Tucker,15,16,5.0,5.0,20.0,21.0,8,5
8,kyle,15,16,4.5,4.5,19.5,20.5,9,7
9,Michael Marzec,12,13,6.5,6.5,18.5,19.5,10,8


### Final Outputs

In [99]:
# Tab 1
prep_results_winner_final

,Name,Expected Correct (per Win%),Expected Correct (per Point Diff),Expected Bonus Points (per Win%),Expected Bonus Points (per Point Diff),Expected Total Score (Per Win%),Expected Total Score (Per Point Diff),Expected Result (per Win%),Expected Result (per Point Diff)
0,Johnny curran,17,17,6.5,6.5,23.5,23.5,1,2
1,Ike Chukwu,14,19,8.0,8.0,22.0,27.0,2,1
2,Danny Swift,17,12,5.0,5.0,22.0,17.0,3,10
3,Michael Hoff,18,18,3.5,3.5,21.5,21.5,4,4
4,Mike Halloran,18,18,3.0,3.0,21.0,21.0,5,6
5,Luke Serra,16,18,5.0,5.0,21.0,23.0,6,3
6,Michael Andrew Ricciardulli,19,13,1.5,1.5,20.5,14.5,7,13
7,Tucker,15,16,5.0,5.0,20.0,21.0,8,5
8,kyle,15,16,4.5,4.5,19.5,20.5,9,7
9,Michael Marzec,12,13,6.5,6.5,18.5,19.5,10,8


In [100]:
# Tab 2
team_tracker_results

,TEAM,Over/Under,Wins,Losses,Win%,Point Diff,Expected Wins (per Win%),Expected Wins (per Point Diff),Expected O/U (per Win%),Expected O/U (per Point Diff)
0,ATL,42.5,8,9,47.1%,2.7,38.6,47.4,Under,Over
1,BRK,37.5,9,9,50.0%,1.6,41.0,45.0,Over,Over
2,BOS,55.5,14,4,77.8%,10.2,63.8,64.1,Over,Over
3,CHO,31.5,6,11,35.3%,-7.9,28.9,23.0,Under,Under
4,CHI,37.5,5,14,26.3%,-7.2,21.6,24.0,Under,Under
5,CLE,50.5,10,9,52.6%,-0.4,43.1,40.0,Under,Under
6,DAL,43.5,11,6,64.7%,1.6,53.1,44.9,Over,Over
7,DEN,52.5,13,6,68.4%,3.8,56.1,50.3,Over,Under
8,DET,27.5,2,17,10.5%,-10.7,8.6,17.2,Under,Under
9,GSW,47.5,8,10,44.4%,0.4,36.4,41.9,Under,Under


In [101]:
# Tab 3
ou_picks_for_tab

,Full name,Boston Celtics,Milwaukee Bucks,Denver Nuggets,Phoenix Suns,Cleveland Cavaliers,Philadelphia 76ers,GS Warriors,LA Lakers,LA Clippers,...,Houston Rockets,Charlotte Hornets,San Antonio Spurs,Portland Trail Blazers,Detroit Pistons,Washington Wizards,Moneyball: Big (3.5),Moneyball: 1-Small (1.5),Moneyball: 2-Small (1.5),Moneyball: 3-Small (1.5)
1,Blake Fahey,Under,Over,Over,Under,Under,Under,Over,Under,Under,...,Under,Under,Under,Over,Over,Over,GS Warriors,Boston Celtics,Milwaukee Bucks,Denver Nuggets
2,Alanna Hulburd,Over,Over,Over,Over,Under,Under,Over,Under,Over,...,Under,Under,Over,Under,Under,Over,GS Warriors,Boston Celtics,Phoenix Suns,Denver Nuggets
3,Michael Hoff,Over,Under,Over,Over,Over,Over,Under,Over,Under,...,Under,Under,Under,Under,Under,Under,Atlanta Hawks,Memphis Grizzlies,Orlando Magic,Indiana Pacers
4,kyle,Over,Over,Over,Under,Over,Under,Over,Under,Under,...,Under,Over,Under,Under,Under,Over,GS Warriors,LA Lakers,Toronto Raptors,OKC Thunder
5,grass guibert,Over,Under,Over,Under,Under,Over,Over,Under,Under,...,Over,Over,Under,Under,Over,Under,GS Warriors,Boston Celtics,Denver Nuggets,Chicago Bulls
6,Johnny curran,Under,Over,Over,Under,Under,Under,Under,Over,Over,...,Over,Under,Under,Under,Over,Over,LA Clippers,Philadelphia 76ers,Memphis Grizzlies,Minnesota Timberwolves
7,Brandon Greene,Over,Over,Under,Over,Under,Under,Over,Over,Under,...,Under,Under,Over,Under,Under,Over,Denver Nuggets,GS Warriors,Portland Trail Blazers,San Antonio Spurs
8,Michael Marzec,Over,Under,Over,Over,Over,Under,Over,Under,Under,...,Under,Over,Under,Under,Under,Over,Minnesota Timberwolves,Phoenix Suns,Detroit Pistons,Utah Jazz
9,Tucker,Under,Over,Over,Over,Under,Over,Under,Under,Over,...,Over,Under,Over,Over,Under,Under,Minnesota Timberwolves,BKN Nets,NO Pelicans,San Antonio Spurs
10,Danny Swift,Over,Over,Over,Under,Over,Over,Over,Under,Under,...,Under,Under,Under,Under,Over,Over,NY Knicks,Philadelphia 76ers,Miami Heat,Washington Wizards
